In [1]:
#Reference -- https://github.com/SAI990323/TALLRec/blob/main/preprocess_movie.py

In [2]:
#Change the current path of the execution
import sys
import os
cwd = f'{os.getcwd()}/../../src/'
sys.path.append(cwd)
os.chdir(cwd)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import json
import numpy as np
import pandas as pd
from subprocess import call

In [5]:
# #Downloading and saving the data
# if os.path.exists(os.path.join('datasets', 'ml-100k')):
#   print("Directory exists!")
# else:
#   print("Directory not exists!")
#   file_path = os.path.join('datasets', 'ml-100k.zip')
#   call(['curl', 'http://files.grouplens.org/datasets/movielens/ml-100k.zip', '-L' ,'-o', file_path])
#   call(['unzip', '-o', file_path, '-d', 'datasets/'])

file_path = os.path.join('datasets', 'ml-100k.zip')
call(['curl', 'http://files.grouplens.org/datasets/movielens/ml-100k.zip', '-L' ,'-o', file_path])
call(['unzip', '-o', file_path, '-d', 'datasets/'])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0Warning: Failed to create the file datasets/ml-100k.zip: No such file or 
  0 4808k    0  4065    0     0   1618      0  0:50:43  0:00:02  0:50:41  1617
curl: (23) Failure writing output to destination
unzip:  cannot find or open datasets/ml-100k.zip, datasets/ml-100k.zip.zip or datasets/ml-100k.zip.ZIP.


9

In [ ]:
#f = open('u.data', 'r')
f = open(os.path.join("datasets", "ml-100k", 'u.data'), 'r')
data = f.readlines()
#f = open('u.item', 'r', encoding='ISO-8859-1')
f = open(os.path.join("datasets", "ml-100k", 'u.item'), 'r', encoding='ISO-8859-1')
movies = f.readlines()
#f = open('u.user', 'r')
f = open(os.path.join("datasets", "ml-100k", "u.user"), 'r')
users = f.readlines()

In [ ]:
movie_names = [_.split('|')[1] for _ in movies] # movie_names[0] = 'Toy Story (1995)'
user_ids = [_.split('|')[0] for _ in users] # user_ids[0] = '1'
movie_ids = [_.split('|')[0] for _ in movies] # movie_ids[0] = '1'

In [ ]:
#Converting data from csv to dictionary
interaction_dicts = dict()  
for line in data:
    user_id, movie_id, rating, timestamp = line.split('\t')
    if user_id not in interaction_dicts:
        interaction_dicts[user_id] = {
            'movie_id': [],
            'rating': [],
            'timestamp': [],
        }
    interaction_dicts[user_id]['movie_id'].append(movie_id)
    interaction_dicts[user_id]['rating'].append(int(int(rating) > 3))
    interaction_dicts[user_id]['timestamp'].append(timestamp)

In [ ]:
interaction_dicts

In [ ]:
#Creating movie id and movie name
with open('datasets/item_mapping.csv', 'w') as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(['movie_id', 'movie_name'])
    for i, name in enumerate(movie_names):
        writer.writerow([i + 1, name])

In [ ]:
#Creating list of lists where each sublist consists of [user_id, movie_ids, ratings, target_movie_id, timestamp]
sequential_interaction_list = []
seq_len = 10
for user_id in interaction_dicts:
    temp = zip(interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'])
    temp = sorted(temp, key=lambda x: x[2]) #-> sorting the map by timestamp
    result = zip(*temp)
    interaction_dicts[user_id]['movie_id'], interaction_dicts[user_id]['rating'], interaction_dicts[user_id]['timestamp'] = [list(_) for _ in result]
    for i in range(10, len(interaction_dicts[user_id]['movie_id'])):
        sequential_interaction_list.append(
            [user_id, 
             interaction_dicts[user_id]['movie_id'][i-seq_len:i], 
             interaction_dicts[user_id]['rating'][i-seq_len:i], 
             interaction_dicts[user_id]['movie_id'][i], 
             interaction_dicts[user_id]['rating'][i], 
             interaction_dicts[user_id]['timestamp'][i].strip('\n')
            ]
        )

In [ ]:
sequential_interaction_list

In [ ]:
#Getting the most recent 10000 sequence interactions from the sequential interactions list
sequential_interaction_list = sequential_interaction_list[-10000:] # 10000 
sequential_interaction_list

In [ ]:
#Creating train, test and validation data
import csv
# save the csv file for baselines
with open('datasets/train.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[:int(len(sequential_interaction_list)*0.8)])
with open('datasets/valid.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.8):int(len(sequential_interaction_list)*0.9)])
with open('datasets/test.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'history_movie_id', 'history_rating', 'movie_id', 'rating', 'timestamp'])
    writer.writerows(sequential_interaction_list[int(len(sequential_interaction_list)*0.9):])

In [ ]:
#Converting csv to json
def csv_to_json(input_path, output_path):
    data = pd.read_csv(input_path)
    json_list = []
    for index, row in data.iterrows():
        row['history_movie_id'] = eval(row['history_movie_id'])
        row['history_rating'] = eval(row['history_rating'])
        L = len(row['history_movie_id'])
        preference = []
        unpreference = []
        for i in range(L):
            if int(row['history_rating'][i]) == 1:
                preference.append(movie_names[int(row['history_movie_id'][i]) - 1])
            else:
                unpreference.append(movie_names[int(row['history_movie_id'][i]) - 1])
        target_movie = movie_names[int(row['movie_id']) - 1]
        preference_str = ""
        unpreference_str = ""
        for i in range(len(preference)):
            if i == 0:
                preference_str += "\"" + preference[i] + "\""
            else:
                preference_str += ", \"" + preference[i] + "\""
        for i in range(len(unpreference)):
            if i == 0:
                unpreference_str += "\"" + unpreference[i] + "\""
            else:
                unpreference_str += ", \"" + unpreference[i] + "\""
        target_preference = int(row['rating'])
        target_movie_str = "\"" + target_movie + "\""
        target_preference_str = "Yes." if target_preference == 1 else "No."
        json_list.append({
            "instruction": "Given the user's preference and unpreference, identify whether the user will like the target movie by answering \"Yes.\" or \"No.\".",
            "input": f"User Preference: {preference_str}\nUser Unpreference: {unpreference_str}\nWhether the user will like the target movie {target_movie_str}?",
            "output": target_preference_str,
        })
        
    with open(output_path, 'w') as f:
        json.dump(json_list, f, indent=4)

# generate the json file for the TALLRec
csv_to_json('datasets/train.csv', 'datasets/train.json')
csv_to_json('datasets/valid.csv', 'datasets/valid.json')
csv_to_json('datasets/test.csv', 'datasets/test.json')

In [ ]:
with open('datasets/train.json', 'r') as lst:
    b = json.load(lst)
    print(b)

In [ ]:
#For reading each sample (Task Instruction + Task Input, Task Output)
for line in b:
    print(line["instruction"])
    print(line["input"])
    print(line["output"])
    print()